In [1]:
from tensorflow.keras.preprocessing import image

def get_data(filename_folder,filename_csv):
    # train.csv 파일 읽어오기
    df = pd.read_csv(filename_csv)
    # 이미지 파일 이름 불러오기
    img_names = df["data"]

    # 이미지를 저장할 ndarray 생성
    images = np.zeros((img_names.shape[0],64,64,3))

    # 이미지 저장
    for index,name in enumerate(img_names):
        images[index] = image.load_img(filename_folder+"/"+name)


    # 레이블 저장
    label = df["label"]

    return images, label

In [2]:
from tensorflow.keras.preprocessing import image

def get_data2(filename_folder,filename_csv):
    # train.csv 파일 읽어오기
    df = pd.read_csv(filename_csv)
    # 이미지 파일 이름 불러오기
    img_names = df["data"]

    # 이미지를 저장할 ndarray 생성
    images = np.zeros((img_names.shape[0],64,64,3))

    # 이미지 저장
    for index,name in enumerate(img_names):
        images[index] = image.load_img(filename_folder+"/"+name)


    # # 레이블 저장
    # label = df["label"]

    return images

In [3]:
def showGraph(hist):
    # 정확률 곡선
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='best')
    plt.grid()
    plt.show()

    # 손실 함수 곡선
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='best')
    plt.grid()
    plt.show()

In [4]:
# 테스트 집합 예측 후 엑셀에 output(label) 저장하기
def result_toExcel(model, x_test):
    res = model.predict(x_test)
    res_real = np.zeros((x_test.shape[0],1))
    res_img_name = np.zeros((x_test.shape[0],1))
    for i in range(x_test.shape[0]):
        for j in range(5):
            if(max(res[i]) == res[i][j]):
                res_real[i] = j

    res_real = res_real.astype(np.uint32)

    df = pd.DataFrame(res_real,columns=['label'])

    filename = input("저장할 파일명 : ")
    filename += ".xlsx"
    df.to_excel(filename,sheet_name='label',index=False)

In [5]:
def result_toExcel2(model, x_test):
    pred=model.predict(x_test)

    pred=list(pred)
    for i in range(len(pred)):
        pred[i] = list(pred[i])
    
    result=[]
    for p in pred:
        result.append(p.index(max(p)))
    
    df=pd.read_csv('submission.csv')
    df['label']=result
    df.to_csv('submission.csv',index=False)

In [6]:
def build_model():
    model=Sequential()
    model.add(Conv2D(32,(3,3),activation='relu',input_shape=(64,64,3)))
    model.add(Conv2D(32,(3,3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    
    model.add(Dense(5,activation='softmax'))
    return model

# 모델 학습 코드

In [8]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 가져오기
images, label = get_data("train","train.csv")

# 훈련집합과 테스트집합으로 분할
x_train,x_test,y_train,y_test=train_test_split(images,label,stratify=label,test_size=0.2)

x_train=x_train.astype(np.float32)/255.0 # 정규화
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,5) # 원핫코드로 변환(레이블은 0,1,2,3,4 총 5개 부류)
y_test=tf.keras.utils.to_categorical(y_test,5)

cnn = build_model()

cnn.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
# 데이터 증대
generator=ImageDataGenerator(zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True)
hist=cnn.fit(generator.flow(x_train,y_train,batch_size=128),epochs=50,validation_data=(x_test,y_test),verbose=2)
# hist=cnn.fit(x_train,y_train,batch_size=128,epochs=30,validation_data=(x_test,y_test),verbose=2)

res=cnn.evaluate(x_test,y_test,verbose=0)
print("정확률은",res[1]*100)

showGraph(hist)

KeyboardInterrupt: 

In [ ]:
# 모델 저장하기
cnn.save('final_final_cnn.h5')

# 저장된 모델을 불러와서 정상적으로 정규화를 한 후 정확률을 측정했을 경우의 결과

In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
import openpyxl
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

cnn=tf.keras.models.load_model("final_final_cnn.h5")

images,label =get_data("test","test.csv")

cnn.summary()

x_test=images.astype(np.float32)/255.0
y_test = tf.keras.utils.to_categorical(label,5)

res=cnn.evaluate(x_test,y_test,verbose=0)
print("정확률은",res[1]*100)



Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-06-14 00:20:35.785078: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-14 00:20:35.785678: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 conv2d_9 (Conv2D)           (None, 60, 60, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 28, 28, 64)        18496     
                                                                 
 conv2d_11 (Conv2D)          (None, 26, 26, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                  

2022-06-14 00:20:37.207398: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-14 00:20:37.298315: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


정확률은 88.06000351905823


# 저장된 모델을 불러온 후 정규화를 하지 않고 정확률을 측정했을 때의 결과

In [8]:
import tensorflow as tf
import numpy as np
import pandas as pd
import openpyxl
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

cnn=tf.keras.models.load_model("final_final_cnn.h5")

images,label =get_data("test","test.csv")

cnn.summary()

y_test = tf.keras.utils.to_categorical(label,5)

res=cnn.evaluate(images,y_test,verbose=0)
print("정확률은",res[1]*100)



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 conv2d_9 (Conv2D)           (None, 60, 60, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 28, 28, 64)        18496     
                                                                 
 conv2d_11 (Conv2D)          (None, 26, 26, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                  

2022-06-14 00:20:58.306793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


정확률은 54.96000051498413


In [16]:
cnn=tf.keras.models.load_model("final_final_cnn.h5")

In [17]:
x_test =get_data2("test","submission.csv")

# predict한 결과를 엑셀파일에 저장할 때 이 부분을 하지 않았습니다...

In [18]:
x_test = images.astype(np.float32)/255 

In [19]:
result_toExcel2(cnn, x_test)

  2/157 [..............................] - ETA: 10s

2022-06-13 11:59:59.250042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


157/157 [==============================] - 2s 9ms/step
